# AN INTRODUCTORY EXAMPLE

In [ ]:
# import macro-functions (libraries) to perform high level computation and funtions

import pandas as pd   # to manage table

import numpy as np    # to manage linear algebra

import matplotlib.pyplot as plt      # to plot data

import seaborn as sns                # to plot data
#sns.set_theme();

import statsmodels.api as sm         # to use statistical tools

from sklearn.preprocessing import scale   # tools to preprocess data

import warnings; warnings.filterwarnings('ignore') # "default" restore default mode "ignore" ignore

## **SCENARIO**: You are working in the marketing department of a car seller. 
## Your boss gives you the last sales data and asked you:
- To create a summary report on the total sales for Manufacturer
- To identify what factors influence the sales
- To make a prediction about how many sales can be generated with an investment of 500k euro in TV Advertising
- To make a prediction about how many sales can be generated with an investment of 75k euro in Social Advertising
- To make a prediction for some new cars models about a chance to win the prestigious award "Car of the Year". Factors of success are Power performance and Fuel Efficiency. Data about these factors for the new cars will be provided

All data are in excel files provided by IT department. 

First I load all the excel file directly in a table

In [ ]:
# ANACONDA NAVIGATOR -- UNCOMMENT HERE
#sales_table = pd.read_csv('./Data/sales_data.csv') # Load a CSV file on a local table (dataframe)

# GOOGLE COLAB - UNCOMMENT HERE

url = 'https://raw.githubusercontent.com/pal-dev-labs/Python-for-Economic-Applications/main/Data/sales_data.csv'
sales_table = pd.read_csv(url)

In [ ]:
# print the dataframe info

sales_table.info()

In [ ]:
# print the dataframe content

sales_table

# DATA EXPLORATION

I want to know how many different models each Manifacturer has

In [ ]:
# for each manufacturer group the models and plot a barplot
sales_table['Manufacturer'].value_counts().plot.bar();

I want to save this image for my future report. I need a nicer figure

In [ ]:
sales_table['Manufacturer'].value_counts().plot.bar()

# add some decorators
plt.xlabel('Manufacturers')
plt.ylabel('Number of Models')
plt.title('Manufacturers different models')
plt.legend()
plt.savefig('manufacturer.png')  # this saves the figure i

# TABLE MANIPULATION

I want to extract total amount of sales for each manufacturer

In [ ]:
total_sales = pd.pivot_table(sales_table, index=['Manufacturer'], values=['Sales_in_thousands'],aggfunc=[np.sum])
total_sales

Let's order a little bit

In [ ]:
total_sales = total_sales.sort_values(by=('sum', 'Sales_in_thousands'), ascending=False)
total_sales

In [ ]:
total_sales.plot.bar(legend = False)
plt.ylabel('Sales (€)');
plt.xlabel('Manufacturers')
plt.title('Summary of Manufacturers total sales')
plt.legend()
plt.savefig('manufacturer_total_sales.png')  # this saves the figure i

In [ ]:
total_sales.iloc[0:15].plot.pie(subplots=True, legend= False, autopct="%1.1f%%")
plt.ylabel('');
plt.xlabel('')
plt.title('Manufacturers Total Sales (k€)')
plt.savefig('manufacturer_total_sales2.png') 

## I would like to understand if there are factors that influence the sales

In [ ]:
plt.scatter(sales_table['Price_in_thousands'].values, sales_table['Sales_in_thousands'].values)
plt.xlabel("Price_in_thousands");plt.ylabel("Sales_in_thousands");

## Let's try with more features

In [ ]:
g = sns.pairplot(sales_table.iloc[:,[3,4,5,6,7,8]]);
g.fig.suptitle("Factors that could influence the sales", fontsize=25)
plt.show()

## Price, TV Advertising (very correlated) and Social Advertising seems interesting

## Let's try to calculate a correlation 

In [ ]:
cor_tv = sales_table['Sales_in_thousands'].corr(sales_table['TV Advert (thousands)'])
cor_social = sales_table['Sales_in_thousands'].corr(sales_table['Social Advert'])
cor_price = sales_table['Sales_in_thousands'].corr(sales_table['Price_in_thousands'])
print("Correlation between Sales and TV Advertising:", cor_tv)
print("Correlation between Sales and Social Advertising:", cor_social)
print("Correlation between Sales and Price:", cor_price)

## TV Advertising seems to be very linearly correlated with Sales.

## Could be interesting to perform a linear regression to have a predictor for sales

$ Sales = \beta_0 + \beta_1 (TVAdv) + \epsilon$

$ y = X \beta + \epsilon$

In [ ]:
# extract X training data from dataframe
X_train = sales_table['TV Advert (thousands)'].values
X_train = sm.add_constant(X_train)

# extract y training data from dataframe
y_train = sales_table['Sales_in_thousands'].values


In [ ]:
X_train

In [ ]:
# set simple linear regression model
model = sm.OLS(y_train,X_train)    # create the OLS model

# train the model
results= model.fit()   # train the model

# Print the model summary
print(results.summary())

In [ ]:
print("Coefficients: ", results.params)
print("R2: ", results.rsquared)

## Let's try to visualize the regression line 

In [ ]:
# ------------------------------------
# calculate predictions
pred_ols = results.get_prediction()

#-------------------------------------
# Confidence intervals
iv_l = pred_ols.summary_frame()["obs_ci_lower"]
iv_u = pred_ols.summary_frame()["obs_ci_upper"]

#--------------------------------------
# plotting
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(X_train[:,1], y_train, "o", label="data")   # plot original data
ax.plot(X_train[:,1], results.fittedvalues, "r--.", label="OLS")  # plot fitted data
ax.plot(X_train[:,1], iv_u, "r--", c='g')   # plot upper confident interval
ax.plot(X_train[:,1], iv_l, "r--", c='g')   # plot lower confident interval

#---------------------------------------
# figure decorator
ax.legend(loc="best")
plt.xlim(0,600)
plt.ylim(0,70)
plt.xlabel('TV Advertising (k euros)')
plt.ylabel('Sales (k)')
plt.show()

## Having a statistical model for TV Advertising I can estimate how many sales can be generated with an investment of 500k euro in TV Advertising

In [ ]:
inv = 500
β1 = results.params[0]  # estimated parameter
β2 = results.params[1]  # estimated parameter
y_pred = β1 + β2 * inv
print("Estimated amount of sales with investment of",inv,"k euros is",y_pred, "k units")

# Let's make the prediction for the TV Advertising

In [ ]:
X_train_social = sales_table['Social Advert'].values

In [ ]:
# train the model

model1 = sm.OLS(y_train,X_train_social)    # create the OLS model
results1 = model1.fit()   # train the model

# Print the model summary
print(results1.summary())

## We got poor results. Let's try with a non linear estimator: KNeighborsRegressor (https://it.wikipedia.org/wiki/K-nearest_neighbors)

In [ ]:
# extract data from original sales dataframe
X_social = sales_table['Social Advert'].values
y = sales_table['Sales_in_thousands'].values

In [ ]:
# import the libraries
from sklearn import neighbors   # to import the model

# preprocess inputs for the model
from sklearn.preprocessing import scale   # to preprocess the input
X_social_prep = scale(X_social, with_mean=False, with_std=False).reshape(-1,1)

# split train test and test test to check accuracy of the model
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_social_prep, y, test_size=0.25)


In [ ]:

# set model hyperparameters
n_neighbors = 20

# set the model
knn = neighbors.KNeighborsRegressor(n_neighbors)

# fit the model
results = knn.fit(X_train, y_train)

In [ ]:
# Calculate the mean squared error (MSE)
def mse(y_true, y_pred):
  return np.mean((y_true - y_pred) ** 2)

# Calculate the test MSE
y_hat_train = results.predict(X_train)
mse_value_train = mse(y_train, y_hat_train)

# Calculate the test MSE
y_hat_test = results.predict(X_test)
mse_value_test = mse(y_test, y_hat_test)

# Print the MSE
print("MSE train:",mse_value_train)
print("MSE test:",mse_value_test)

# Note that the MSE change each time we run the model

## Let's have a look of the predition

In [ ]:
y_hat = results.predict(X_social_prep)

plt.scatter(X_social_prep, y, color='darkorange', label='data')
plt.scatter(X_social_prep, y_hat, color='navy', label='prediction')
plt.legend()
plt.title("KNeighborsRegressor (k = %i)" % (n_neighbors))

plt.tight_layout()
plt.xlim(0,200000)
plt.ylim(0,400)
plt.xlabel("Social Advertising investment (euros)")
plt.ylabel("Sales (k units)")

plt.show()

## Do we have the best model? We can change the hyperparameters **n_neighbors** and check MSE

In [ ]:
# split train test and test test to check accuracy of the model
X_train, X_test, y_train, y_test = train_test_split(X_social_prep, y, test_size=0.25)

# we will use the SAME set to train the model with different hyperparameters
mse_value_train = []
mse_value_test = []

for i in range(1,100):
    n_neighbors = i
    knn = neighbors.KNeighborsRegressor(n_neighbors)
    results = knn.fit(X_train, y_train)
    # Calculate the test MSE
    y_hat_train = results.predict(X_train)
    mse_value_train.append(mse(y_train, y_hat_train))
    # Calculate the test MSE
    y_hat_test = results.predict(X_test)
    mse_value_test.append(mse(y_test, y_hat_test))

In [ ]:
plt.plot(mse_value_train, label='Train MSE')
plt.plot(mse_value_test, label='Test MSE')
plt.legend()
plt.xlabel('Neighbors')
plt.ylabel('MSE')

## we can take neighbors = 50 as best hyperparameter and make a prediction for the investment of 75k euro in Social Advertising


In [ ]:
## we can take neighbors = 50 as best hyperparameter and prediction about and make a prediction for the investment of 75k euro in Social Advertising
n_neighbors = 50

knn = neighbors.KNeighborsRegressor(n_neighbors)
results = knn.fit(X_train, y_train)
y_hat = results.predict(np.array([75000]).reshape(-1,1))

print('An investment of 75k euro in Social Advertising generates ',y_hat[0], 'units of sale')

# Last question: The award

## We have to make a prediction for three new cars models about a chance to win the prestigious award "Car of the Year".

## Factors of success are Power performance and Fuel Efficiency. 

In [ ]:
sales_table[['Manufacturer','Model','Fuel_efficiency','Power_perf_factor','Awarded']]

## We note that the column "Awarded" has 0 for NOT win and 1 for WIN.

## We have to build a standard classifier with 2 predictors: "Fuel_efficiency" and "Power_perf_factor"

## Let's load the file provided by IT

In [ ]:
# Load the CSV file provided by IT on a local table (dataframe)
award_factors = pd.read_csv('./Data/award-factors.csv')

# Alternatively load the file from an url
#url = 'https://raw.githubusercontent.com/pal-dev-labs/Python-for-Economic-Applications/main/Data/award-factors.csv'
#award_factors = pd.read_csv(url)

In [ ]:
award_factors

## We will use a Neural Network as a classifier. We will use 2 preditors 'Fuel_efficiency' and 'Power_perf_factor'

In [ ]:
# Split the data into features and target
X = sales_table[['Fuel_efficiency', 'Power_perf_factor']]
y = sales_table['Awarded']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [ ]:
from tensorflow import keras

# Define the model
model = keras.Sequential([
  keras.layers.Dense(16, activation='relu', input_shape=(2,)),
  keras.layers.Dense(32, activation='relu'),
  keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model on the training set
model.fit(X_train, y_train, epochs=100);

In [ ]:
# Evaluate the model on the testing set
loss, accuracy = model.evaluate(X_test, y_test)

# Print the loss and accuracy
print('Loss:', loss)
print('Accuracy:', accuracy)

## Make predictions on new cars


In [ ]:
# Make predictions on new data

predictions = model.predict(award_factors[['Fuel_efficiency', 'Power_perf_factor']])
award_factors['Probability to Win (%)']=np.around(predictions*100,2)

award_factors

# PYTHON PROGRAMMING LANGUAGE

## Everything Is an Object
Python is an **object-oriented programming language (OOPP)** and in Python everything is an **OBJECT**.

In an object-oriented programming languages like Python, an object is an entity that contains data along with associated metadata and/or functionality. 

In Python **everything is an object**, which means every entity has some metadata (called **attributes** or **fields**) and associated functionality (called **methods**). These attributes and methods are accessed via the dot syntax.

Example of objects are the integers numbers **1,2,3** or the symbols **"a"**,**"B"**,**"?"**. A "bigger" object could be a container of numbers **[1,2,3,4...,9,..,101]**

As objects are so fundamental, Python has already some built-in objects, like numbers and characters

Let's create an object **a**

In [ ]:
"a"

We use the characters "" to create the object **'a'** whose data is the symbol *a*

Let's create an object that is a number

In [ ]:
3

the object **3** contains the symbol 3 (that python considers as the mathematical value 3

Consider now

In [ ]:
"3"

The object **'3'** is different from the object **3**. The last is the character *3* the first is the mathematical value 3

Different objects behave differently when we apply operations

In [ ]:
3+3

In [ ]:
"3"+"3"

Objects contain not only data or information but also **fields** and **methods**. We can use **dot** notation to access fields and methods

For example **capitalize** is a method to capitalize a symbol contained in a character object. It **produces** a new object that is the capitalized character


In [ ]:
# capitalize is a method to capitalize the symbol a contained in the object 'a'. It produces the new object 'A'
"a".capitalize()

In [ ]:
"A".lower()

The following picture summarize the objects creation process. We can note that every objects has a **TYPE** (or we can say it belongs to a **CLASS**)

https://github.com/pal-dev-labs/Python-for-Economic-Applications/blob/main/Images/objects1.png

# Types
Objects have type information attached. 

There are built-in simple types offered by Python and several compound types, which will be discussed in the following lessons.

Python's simple types are summarized in the following table:

<center>**Python Scalar Types**</center>

| Type        | Example        | Description                                                  |
|-------------|----------------|--------------------------------------------------------------|
| ``int``     | ``1``      | integers (i.e., whole numbers)                               |
| ``float``   | ``1.0``    | floating-point numbers (i.e., real numbers)                  |
| ``complex`` | ``1 + 2j`` | Complex numbers (i.e., numbers with real and imaginary part) |
| ``bool``    | ``True``   | Boolean: True/False values                                   |
| ``str``     | ``'Ab$'``  | String: characters or text                                   |
| ``NoneType``| ``None``   | Special object indicating nulls                              |

We'll take a quick look at each of these in turn.

In [ ]:
type(3)

In [ ]:
type("Ab$")

In [ ]:
type(1.5)

In [ ]:
type(3+3)

In [ ]:
type("abc"+"cde")

## Python Variables
Let's see how python manages variables.

We're going to assign the int value *4* to a variable named *x*, the str value "Stefano" to a variable named *name1* and a float *12.8* value to a variable named *c*

In [ ]:
# assignment instructions

x = 4     # assign 4 to variable x
name1 = "Stefano"   #   # assign 'Stefano' to variable name1
c = 12.8 # assign 12.8 to variable c


## Python Variables Are Pointers

Assigning variables in Python is as easy as putting a variable name to the left of the equals (``=``) sign:

```python
# assign 4 to the variable x
x = 4
```

It seems as we create a space in memory, named x, and insert directly the value 4 in that space.

This is NOT the way in which Python works.

In Python variables are best thought of not as containers but as **POINTERS**.
So in Python, when you write

```python
x = 4
```

you are essentially defining a *pointer* named ``x`` that points to an object in memory that contains the value ``4``. The right part of the assignment instruction above, creates an int object in memory and assignes the address memory of that object to the pointer ``x``. 


In this way, variable a is able to accesso all the information of the object, including value, fields and methods.


https://github.com/pal-dev-labs/Python-for-Economic-Applications/blob/main/Images/fig-variable-object-3.png

In [ ]:
type(x)

This works also for more complex objects


In [ ]:
L = [1,2,3,4,5,6,7,8]        # create a container (a list) that contains the number 1,2,3..8 and assign the address of the object to the pointer L
print('We create the variable (pointer) L that points to the object',L,'which is of kind ',type(L))

The above instruction is an ASSIGNMENT instruction.

In the left side of the = we create in memory the object of kind list (a container) that contains the value 1,2,3,4,5,6,7,8

In the righe side of the = we create a pointer L that CONTAINS ONLY THE MEMORY ADDRESS of the object 


https://github.com/pal-dev-labs/Python-for-Economic-Applications/blob/main/Images/fig-variable-object-6.png


When we create an object, we create not only values but also some functions (methods) associated with objects. 

With the pointers we can access these functions with the **dot** notation

In [ ]:
a = "this is a Python course"
print(a)    # print the value of the object

In [ ]:
type(a)

an object of kind STR has many methods and we can call them with the notation a. 

For example

In [ ]:
a.capitalize()

In [ ]:
a.split()

We can see all methods associate with an object with a.[TAB]

Consider now

In [ ]:
y = [1, 2, 3]   # this create a container with 3 numbers
x = y

We've created two variables ``x`` and ``y`` which both point to the same object.
Because of this, if we modify the list via one of its names, we'll see that the "other" list will be modified as well:

In [ ]:
print(y)

In [ ]:
print(x)

In [ ]:
x.append(4) # append 4 to the list pointed to by x
print(y) # y's list is modified as well!

This behavior might seem confusing if you're wrongly thinking of variables as buckets that contain data.
But if you're correctly thinking of variables as pointers to objects, then this behavior makes sense.

Note also that if we use "``=``" to assign another value to ``x``, this will not affect the value of ``y`` – assignment is simply a change of what object the variable points to:

In [ ]:
x = 'something else'
print(y)  # y is unchanged

Again, this makes perfect sense if you think of x and y as pointers, and the "=" operator as an operation that changes what the name points to.

You might wonder whether this pointer idea makes arithmetic operations in Python difficult to track, but Python is set up so that this is not an issue. Numbers, strings, and other simple types are IMMUTABLE: you can't change their value – you can only change what values the variables point to. So, for example, it's perfectly safe to do operations like the following:

In [ ]:
x = 10 
y = x
x = x+5  # we're creating a new object that contains the value 15
print("x =", x)
print("y =", y)

When we call ``x = x+5``, we are not modifying the value of the ``10`` object pointed to by ``x``; we are rather changing the variable ``x`` so that it points to a new integer object with value ``15``.
For this reason, the value of ``y`` is not affected by the operation.

## Arithmetic Operations (between objects of type numbers)
Python implements seven basic binary arithmetic operators, two of which can double as unary operators.
They are summarized in the following table:

| Operator     | Name           | Description                                            |
|--------------|----------------|--------------------------------------------------------|
| ``a + b``    | Addition       | Sum of ``a`` and ``b``                                 |
| ``a - b``    | Subtraction    | Difference of ``a`` and ``b``                          |
| ``a * b``    | Multiplication | Product of ``a`` and ``b``                             |
| ``a / b``    | True division  | Quotient of ``a`` and ``b``                            |
| ``a // b``   | Floor division | Quotient of ``a`` and ``b``, removing fractional parts |
| ``a % b``    | Modulus        | Integer remainder after division of ``a`` by ``b``     |
| ``a ** b``   | Exponentiation | ``a`` raised to the power of ``b``                     |
| ``-a``       | Negation       | The negative of ``a``                                  |
| ``+a``       | Unary plus     | ``a`` unchanged (rarely used)                          |

These operators can be used and combined in intuitive ways, using standard parentheses to group operations.
For example:

In [ ]:
# addition, subtraction, multiplication
(4 + 8) * (6.5 - 3)

## Comparison Operations

Another type of operation which can be very useful is comparison of different values.
For this, Python implements standard comparison operators, which return Boolean values ``True`` and ``False``.
The comparison operations are listed in the following table:

| ``a == b``| ``a`` equal to ``b``      
| ``a != b`` | ``a`` not equal to ``b``             
| ``a < b``| ``a`` less than ``b``         
| ``a > b``| ``a`` greater than ``b``             
| ``a <= b``| ``a`` less than or equal to ``b``
|``a >= b`` | ``a`` greater than or equal to ``b``



These comparison operators can be combined with the arithmetic and bitwise operators to express a virtually limitless range of tests for the numbers.
For example, we can check if a number is odd by checking that the modulus with 2 returns 1:

In [ ]:
2 < 1

In [ ]:
# 25 is odd
25 % 2 == 1

In [ ]:
# check if a is between 15 and 30
a = 25
15 < a < 30

## Boolean Operations
When working with Boolean values, Python provides operators to combine the values using the standard concepts of "and", "or", and "not".
Predictably, these operators are expressed using the words ``and``, ``or``, and ``not``:

In [ ]:
x = 4
(x < 6) and (x > 2)

In [ ]:
(x > 10) or (x % 2 == 0)

In [ ]:
not (x < 6)

Boolean algebra aficionados might notice that the XOR operator is not included; this can of course be constructed in several ways from a compound statement of the other operators.
Otherwise, a clever trick you can use for XOR of Boolean values is the following:

In [ ]:
# (x > 1) xor (x < 10)
(x > 1) != (x < 10)

These sorts of Boolean operations will become extremely useful when we begin discussing *control flow statements* such as conditionals and loops.

One sometimes confusing thing about the language is when to use Boolean operators (``and``, ``or``, ``not``), and when to use bitwise operations (``&``, ``|``, ``~``).
The answer lies in their names: Boolean operators should be used when you want to compute *Boolean values (i.e., truth or falsehood) of entire statements*.
Bitwise operations should be used when you want to *operate on individual bits or components of the objects in question*.

## A Container object: the type LIST

In [ ]:
"""List type."""

# Lists can contain any type of variable and they can contain as many variables as you wish.
# Lists can also be iterated over in a very simple manner.
# Here is an example of how to build a list.
squares = [1, 4, 9, 16, 25]

In [ ]:
print(squares)

In [ ]:
type(squares)

In [ ]:
# Lists can be indexed and sliced:
print(squares[0])  # indexing returns the item
print(squares[0:3]) # slicing returns a new list
print(squares[-1]) # start from the last element

# All slice operations return a new list containing the requested elements.


In [ ]:
# Lists also support operations like concatenation:
print(squares + [36, 49, 64, 81, 100])


In [ ]:
# Unlike strings, which are immutable, lists are a mutable type, i.e. it
# is possible to change their content:
cubes = [1, 8, 27, 65, 125]  # something's wrong here, the cube of 4 is 64!
cubes[3] = 64  # replace the wrong value
print( cubes )

## LIST methods

In [ ]:
fruits = ['orange', 'apple', 'pear', 'banana', 'kiwi', 'apple', 'banana']

# list.append(x)
# Add an item to the end of the list.
# Equivalent to a[len(a):] = [x].
fruits.append('grape')
print( fruits)

In [ ]:
# list.remove(x)
# Remove the first item from the list whose value is equal to x.
# It raises a ValueError if there is no such item.
fruits.remove('grape')
print( fruits )

In [ ]:
# list.insert(i, x)
# Insert an item at a given position. The first argument is the index of the element
# before which to insert, so a.insert(0, x) inserts at the front of the list,
# and a.insert(len(a), x) is equivalent to a.append(x).
fruits.insert(0, 'grape')
print( fruits )


In [ ]:
# list.sort(key=None, reverse=False)
# Sort the items of the list in place (the arguments can be used for sort customization,
# see sorted() for their explanation).
fruits.sort()
print( fruits )


In [ ]:
# list.pop([i])
# Remove the item at the given position in the list, and return it. If no index is specified,
# a.pop() removes and returns the last item in the list. (The square brackets around the i in
# the method signature denote that the parameter is optional, not that you should type square
# brackets at that position.)
print( fruits )
print( fruits.pop() )
print( fruits )


In [ ]:
# list.clear()
# Remove all items from the list.
fruits.clear()
print( fruits)

## PYTHON FUNCTIONS
In Python a function is a particular OBJECT that is able to receive in input one or more parameters, perform some actions and produce an output.
There are 2 ways to create a function: with a definition, by the mean of the anonymous functions

## Create a function by definition
Let's try to create $f(x) = 2x + 4$


In [ ]:
def f(x):    # we use the 'def' command followed by the name of the function; we also use () to specify input variables
    y = 2 * x + 4  # we specify the form of the function
                   # TAKE CARE: instructions are indented. It's python sintax to create a block of code
    return y

In [ ]:
f(7)

In [ ]:
# Let's define functions with more variables
def g(x,y):
    z = 2 * x-3 *y
    return z

In [ ]:
g(4,9)

The nice thing in Python is that it manages automatically the type of variable 

In [ ]:
def sumvariable(x,y):
    z = x+y
   # print("The result of the operation is: ",z)
    return z       # return is mandatory when you have to give back to the caller some results

result = sumvariable(5,3)
print("Result is: ", result)

In [ ]:
sumvariable(2,3)

In [ ]:
sumvariable("ciao","hello")

Take care: objects created inside a function "Leaves" only there. You cannot access the value of internal variables from the outside (SCOPING)

# EXERCISE
- Define a function that receive a list of integer as input and return the same list, sorted from the lower to the higher

## ANONYMOUS FUNCTIONS
We can create a function in a quicker way be means of the Anonymous Functions.  
As everything in Python, Anonymous functions are also OBJECTS which can be referenced by a variable.  
The instruction to create it is:  

In [ ]:
# we associate to the input x the form 2x+3
lambda x: 2*x+3

In [ ]:
print("The type of 'lambda x: 2*x+3' is",type(lambda x: 2*x+3))

Note that we created just an object of type function. We do not have a pointer to that object

Python is a first-class functions programming language as it treats functions as first-class citizens (objects).

We can create the pointer assigning the function to a variable. Better we can reference an object function with a variable.


In [ ]:
f1 = lambda x: 2*x+3
# now we can call the funtion with the variable using ()
f1(2)

First-class functions are a necessity for the functional programming style, in which the use of higher-order functions is a standard practice

Treating functions as objects gives the chance to define a function that has as input parameter, a function (high order functions)

In [ ]:
# Let's create a custom high order function: map2
# note as in python we don't need to specify that there is a function in the input parameter

def map2(function1,input1):  # this function has the first input parameter that is a function
    temp = function1(input1)  # we use the function notation here, using ()
    return temp

In [ ]:
map2(lambda x: 4*x,2)  # we call the high order function map2 with a function as parameter

A practical example: the use of the built in high order function **map**

In [ ]:

a=[1,2,3,4,5]
# we use the built in function map that takes as first argument a FUNCTION 
# take care of the anonymous function we use in it

b=map(lambda x: x*4,a)
list(b)   # function list is necessary to print b

import requests
from IPython.display import Image# EXERCISE

Try to execute all the cells of the notebook

# EXERCISES
- Use map function to square each element of a list of integer numbers
- Use map function to convert from Celsius degree to Fahrenheit each element of a list of Celsius Temperatures
    (1 fahrenheit = celsius * 9/5 +32
- Use filter function (similar as map) to filter even element of a list of numbers



## Solutions

In [ ]:
print(list(map(lambda x: x*2, [1,2,3,4])))

In [ ]:
def celsius_to_fahrenheit(celsius):
    return celsius * 9 / 5 + 32

celsius_temperatures = [10, 20, 30, 40, 50]

fahrenheit_temperatures = list(map(celsius_to_fahrenheit, celsius_temperatures))

print(fahrenheit_temperatures)

In [ ]:
def is_even(x):
    return x % 2 == 0

numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

even_numbers = list(filter(is_even, numbers))

print(even_numbers)